In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from sentence_transformers import SentenceTransformer, util

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model = SentenceTransformer("jhgan/ko-sbert-nli")

In [3]:
df = pd.read_csv("../../data/train_organized.csv")

In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
# ✅ 2. 예시 데이터 구성


data = []
for fname, dial, summary, topic in zip(df['fname'], df['dialogue'], df['summary'], df['topic']):
    x = dial.strip().replace("\n", "")
    lines = re.split(r"(?=#person\d+#:)", x, flags=re.IGNORECASE)
    if lines and lines[0].strip() == "":
            lines = lines[1:]

    data.append({
      "dialogue": lines,
      "summary": summary,
      "topic": topic,
      "fname": fname
    })

# ✅ 3. 유사도 계산 및 랭킹
results = []

for idx, item in enumerate(data):
    dialogue = item["dialogue"]
    summary = item["summary"]
    topic = item["topic"]
    fname = item["fname"]

    # 임베딩
    sentence_embeddings = model.encode(dialogue, convert_to_tensor=True)
    summary_embedding = model.encode(summary, convert_to_tensor=True)

    # 코사인 유사도
    cosine_scores = util.cos_sim(sentence_embeddings, summary_embedding).squeeze(-1)

    # 결과 정리
    df = pd.DataFrame({
        "dialogue_id": idx,
        "summary": summary,
        "dialogue": dialogue,
        "cosine_similarity": cosine_scores.cpu().numpy(),
        "topic": topic,
        "fname": fname
    })
    df["rank"] = df["cosine_similarity"].rank(ascending=False, method="first").astype(int)
    results.append(df)

# ✅ 4. 최종 통합 결과
final_df = pd.concat(results, ignore_index=True)
final_df.to_csv("sbert_dial_result.csv", index=False)

In [13]:
def extract_high_similarity_texts(df, data_len, threshold=0.3):
    """
    각 dialogue_id 별로 cosine_similarity가 threshold 이상인 문장만 선택하여
    'n'으로 구분된 문자열로 만든 후 리스트로 반환

    Parameters:
        df (pd.DataFrame): 'dialogue_id', 'cosine_similarity', 'dialogue' 컬럼 포함
        data_len (int): 전체 데이터 개수
        threshold (float): 유사도 임계값

    Returns:
        List[str]: 'n'으로 구분된 중요 문장 문자열 리스트
    """
    result = {
      "topic": [],
      "fname": [],
      "dialogue": [],
      "summary": []
    }

    for i in range(data_len):
        target = df[df["dialogue_id"] == i]
        
        if len(target) <= 5:
          continue
        
        # 유사도 threshold 이하 문장 제거
        filtered = target[target["cosine_similarity"] >= threshold]

        filtered = filtered.dropna(subset=["dialogue"])

        if len(target) - len(filtered) <= 2:
          continue

        if len(filtered) == 0:
          print(f"target: {target}")
          continue
        
        # 남은 문장들을 "n"으로 조인
        joined_text = "\n".join(filtered["dialogue"].tolist())
        # 이제 진짜로 join된 텍스트가 비어있지 않은지 최종 확인
        if joined_text.strip() == "" or joined_text.strip() == "nan":
            print(f"[EMPTY AFTER JOIN] dialogue_id: {i}")
            continue

        result["topic"].append(target["topic"].iloc[0])
        result["fname"].append(target["fname"].iloc[0])
        result["dialogue"].append(joined_text)
        result["summary"].append(target["summary"].iloc[0])

    return result

In [14]:
filtered_datas = extract_high_similarity_texts(final_df, data_len=len(data), threshold=0.36)

target:        dialogue_id                              summary  \
56102         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56103         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56104         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56105         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56106         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56107         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56108         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   
56109         5898  피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.   

                                                dialogue  cosine_similarity  \
56102                                #Person1#: 안녕, 모니카.           0.355420   
56103                                 #Person2#: 안녕, 피터.           0.275758   
56104  #Person1#: 수요일 밤에 뭐 해? 글로벌 시어터에서 멋진 쇼가 있어. 같이 갈래?           0.338077   
56105                #Person2#: 수요일 밤? 아, 미안, 작문을 끝내야 해.           0.291577   
56106  #Person1#: 아, 그래? 괜찮아. 그럼 목요일은 어때? 우리 저녁 먹으러 나갈까?         

In [15]:
filtered_df = pd.DataFrame(filtered_datas)
filtered_df.to_csv("sbert_filtered_dial.csv", index=False)

In [16]:
display(len(filtered_df))

3464